In [12]:
import orjson
pwd = '~/projects/gun-overlays/bible/'

In [13]:
with open(pwd+'/roh.json') as f:
    data = orjson.loads(f.read())

for book in data['books']:
    with open(pwd+'/roh/'+book['abbreviation']+'.json', 'wb') as f:
        f.write(orjson.dumps(book.pop('chapters'), option=orjson.OPT_INDENT_2))

with open(pwd+'/roh/index.json', 'wb') as f:
    f.write(orjson.dumps(data, option=orjson.OPT_INDENT_2))

In [15]:
with open(pwd+'/seb.json') as f:
    data = orjson.loads(f.read())

for book in data['books']:
    with open(pwd+'/seb/'+book['abbreviation']+'.json', 'wb') as f:
        f.write(orjson.dumps(book.pop('chapters'), option=orjson.OPT_INDENT_2))

with open(pwd+'/seb/index.json', 'wb') as f:
    f.write(orjson.dumps(data, option=orjson.OPT_INDENT_2))

# Scrape biblia.sk

In [4]:
from time import time, sleep
import binascii
import json
import requests

res = requests.get('https://api2.biblia.sk/api/preklady')
preklady = res.json()['data']
with open('preklady.json', 'w') as f:
    json.dump(preklady, f, indent=2, ensure_ascii=False)

for identifier in ['bot']:  #'seb','sep','roh','ssv',
    preklad = None
    for p in preklady:
        if p["identifier"] == identifier:
            preklad = p
            break
    if preklad is None:
        print("Identifier not found", identifier)
        continue
    books = preklad['books']

    # params = {
    #     'timestamp': int(time()*1000),
    #     'key': '597133743677397A24432646294A404E635166546A576E5A7234753778214125',
    # }
    for i, book in enumerate(books):
        chapters = {}
        abbr = book['abbreviation']
        for chapter in book['chapters']:
            # params['timestamp'] = int(time()*1000)
            # addr = f"{identifier}|{abbr}|{chapter}"
            # code = binascii.b2a_base64(addr.encode()).strip().decode()
            while True:
                res = requests.get(f"https://biblia.sk/api/chapter?translation={identifier}&book={abbr}&chapter={chapter}")
                print(res.status_code, res.url[:70])
                if res.status_code == 429:
                    print('Too Many Requests, sleeping')
                    sleep(40)
                    continue
                break
            try:
                text = res.json()
            except Exception as e:
                print(res.content)
                raise e
            chapters[str(chapter)] = [v['search_text'].strip() for v in text['data']]
        book['chapters'] = chapters
        # Write book.json
        with open(f"{identifier}/{abbr}.json", "w") as f:
            json.dump(chapters, f, indent=2, ensure_ascii=False)
    break


200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=1
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=2
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=3
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=4
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=5
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=6
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=7
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=8
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=9
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=10
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=11
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=12
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=13
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=14
200 https://biblia.sk/api/chapter?translati

In [8]:
# Create routes/bible/{bible}.json.js
import json
with open('../../src/routes/bible/seb.json.js') as f:
    seb = json.loads(f.read()[21:-66])
sebaliases = {b["abbreviation"]: b["aliases"] for b in seb["books"] if "aliases" in b}
with open('preklady.json') as f:
    preklady = json.load(f)

for identifier in ['ssv', 'bot']:
    preklad = None
    for p in preklady:
        if p["identifier"] == identifier:
            preklad = p
            break
    for book in preklad["books"]:
        book.pop("chapters")
        try:
            book["aliases"] = sebaliases[book["abbreviation"]]
        except KeyError:
            pass
    with open(f'../../src/routes/bible/{identifier}.json.js', 'w') as f:
        f.write('export const index = ')
        json.dump(preklad, f, indent=2, ensure_ascii=False)
        f.write(""";

    export async function get() {
      return {
        body: index,
      }
    }""")


In [11]:
# Correct order of books
with open(f'../../src/routes/bible/seb.json.js') as f:
    seb = json.loads(f.read()[21:-66])

for identifier in ['ssv', 'bot']:
    with open(f'../../src/routes/bible/{identifier}0.json.js') as f:
        bible = json.loads(f.read()[21:-66])
    reordered_books = []
    for book in seb["books"]:
        for b in bible["books"]:
            if b["abbreviation"] == book["abbreviation"]:
                reordered_books.append(b)
                break
    bible["books"] = reordered_books
    with open(f'../../src/routes/bible/{identifier}.json.js', 'w') as f:
        f.write('export const index = ')
        json.dump(bible, f, indent=2, ensure_ascii=False)
        f.write(""";

    export async function get() {
      return {
        body: index,
      }
    }""")